In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd

# 함수 정의
# 화면 스크롤 하는 함수
def scroll_to(height):
    current_scroll_position = driver.execute_script("return window.pageYOffset;")       # 현재 위치 가져오기
    new_scroll_position = current_scroll_position + height          # 지금 위치에서 height 추가한 새로운 스크롤 위치
    driver.execute_script(f'window.scrollTo(0,{new_scroll_position});')     # 새로운 스크롤 위치로 이동
    time.sleep(1)

# 지역 버튼 클릭하는 함수
def click_region(location_number):
    # 전남 id = 38 | 서울 id = 1 | 대구 id = 4
    button_path = driver.find_element(By.XPATH, f'//*[@id="{location_number}"]/button')
    button_path.click()
    time.sleep(2)

tour = []

service=webdriver.chrome.service.Service('chromedriver.exe')
driver=webdriver.Chrome(service=service)


driver.get('https://korean.visitkorea.or.kr/list/travelinfo.do?service=ms#ms^0^32^All^e6875575-2cc2-43ba-9651-28d31a7b3e23,651c5b95-a5b3-11e8-8165-020027310001,c24d515f-3202-45e5-834e-1a091901aeff,d3fd4d9f-fbd4-430f-b5d5-291b4d9920be,3f36ca4b-6f45-45cb-9042-265c96a4868c,23bc02b8-da01-41bf-8118-af882436cd3c,2d4f4e06-2d37-4e54-ad5c-172add6e6680,9668f0f1-8afe-4526-8007-503bd02fd6d8,0f29b431-75ac-4ab4-a892-b247d516b31d,640d3489-8fc3-11e8-8165-020027310001,1601b0a3-144e-40b7-95b4-b946e537a25b,1c981ad4-7834-11e8-82c8-020027310001,266bf7a0-cbab-4bb4-b800-d7edd5642180,cdd12d65-1f38-4829-a1be-bf235a0fb3f2^1^^1^#%EA%B0%95%EC%9B%90')
time.sleep(5)

# 지역 번호 바꾸세요!!!!
region_num = 4
click_region(region_num)

# 전체 결과 수 확인하고 총 페이지 수 확인
total_results = int(driver.find_element(By.XPATH,'//*[@id="totalCnt"]').text.replace(",",""))
total_pages = total_results//10 +1     # 한 페이지당 10개씩 나와요 그래서 총 페이지 수 total_results // 10 + 1 해줘야함


# 페이지마다 결과물 10개씩 나옴
# 그거 다 받아서 for 문으로 돌리면서 하나씩 들어가서 정보 crawling
for _ in range(total_pages):
    # 해당 페이지에 보여지는 결과물 10개 contents에 저장
    
    location_counts = len(driver.find_elements(By.CSS_SELECTOR, '#contents > div:nth-child(2) > div:nth-child(1) > ul > li'))
    # 결과물 1개씩 클릭해서 정보 크롤링
    for idx in range(location_counts):
        # 정보 저장할 딕셔너리 초기화
        location_data = {}
        # 장소 이름 클릭할 수 있도록 button 찾아주고 클릭
        time.sleep(2)
        locations = driver.find_elements(By.CSS_SELECTOR, '#contents > div:nth-child(2) > div:nth-child(1) > ul > li')
        link = locations[idx].find_element(By.TAG_NAME, 'a')
        link.click()
        time.sleep(2)

        # 페이지로 이동한 후 데이터 얻기
        # 장소 이름 저장
        title = driver.find_element(By.ID, 'topTitle')
        location_data['place'] = title.text

        # 상세 정보 보이는 쪽으로 스크롤
        scroll_to(900)
        time.sleep(1)

        # 상세정보 찾아서 text 저장
        info = driver.find_element(By.CSS_SELECTOR, 'div.inr_wrap > div.inr > p')
        location_data['info'] = info.text
        time.sleep(1)

        # 상세 항목 저장
        # 아래쪽으로 이동
        scroll_to(600)
        # 더보기 있으면 클릭
        try:
            driver.find_element(By.XPATH, '//*[@id="detailinfoview"]/div/div[2]/button').click()
        except:
            pass
        
        # 장소 상세 정보 다운로드
        additional_contents = driver.find_elements(By.CSS_SELECTOR, 'div.wrap_contView div.inr li')
        for additional_content in additional_contents:
            key = additional_content.find_element(By.CSS_SELECTOR, 'strong').text
            value = additional_content.find_element(By.CSS_SELECTOR, 'span').text
            location_data[key] = value
        
        # 리뷰 데이터 크롤링
        review_list = []    # 리뷰 담을 변수 초기화
        scroll_to(600)
        try:
            review_count = int(driver.find_element(By.ID, 'commentCount').text)

            # 리뷰 개수 세기
            count = min( 10, review_count)
            # 리뷰 개수 0개이면 뒤로가기 
            if count == 0 :
                driver.back()
                continue
            # 리뷰 2개 이하면 리뷰 추출 (더보기 안 눌러도 다 보여서 굳이 더보기 누를 필요 없음)
            elif count <=2:
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
            # 리뷰 3개이상 7개 이하면 더보기 한 번 눌러야함
            elif count <=7:
                scroll_to(300)
                driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click() # 더보기 버튼 누르기     # 리뷰 더보기 클릭
                time.sleep(2)
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
            # 리뷰 8개 이상이면 더보기 2번 눌러야함
            else:
                scroll_to(300)
                driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
                time.sleep(1)
                scroll_to(700)
                try:
                    driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
                except:
                    pass
                review_contents = driver.find_elements(By.CSS_SELECTOR, '.commentArea li')
            
            scroll_to(200)

            # 리뷰 컨텐츠들을 하나씩 꺼내어서 review_text 추출
            for review_content in review_contents:
                scroll_to(187)
                review_list.append(review_content.find_element(By.CLASS_NAME, 'review_text').text)
            reviews = " ".join(review_list)

            # 리뷰 정보 딕셔너리에 저장
            location_data['review'] = reviews

        except: # 오류 발생시 오류라고 저장하고 넘어가게 해줌
            location_data['review']= '오류'

        finally:
            # 투어 리스트에 location_data 전체 저장
            tour.append(location_data)
            print(location_data)

            # 정보 다 크롤링한 후 뒤로 가기
            driver.back()
            scroll_to(120)
            time.sleep(2)

    # 아래로 내려가기
    scroll_to(200)
    time.sleep(1)

    # 다음 페이지 버튼 누르기
    page_button_path = driver.find_elements(By.CSS_SELECTOR, '.page_box > a')
    page_button_path[-2].click()
    time.sleep(3)

pd.DataFrame(tour).to_csv(f'{region_num}번 지역 데이터')
time.sleep(1)
driver.close()

{'place': '화산산성', 'info': '화산산성은 해발 828m 화산 정상부에 외부로부터 침입을 막기 위해 조선 후기에 쌓았다. 둘레는 약 9,300보, 높이가 10척으로 기록되어 있다. 조선 숙종 35년(1709)에 병마절도사 윤숙이 군대를 주둔시키기 위해 동쪽과 서쪽, 남쪽과 북쪽 사방 문의 기초 공사를 시작하였다. 성벽을 쌓기 위해 돌로 된 재료를 모으고 수구문을 쌓던 중 흉년과 질병이 겹쳐 공사가 중지되었고, 그 후 계속 지금의 상태로 두었다고 한다. 조선 중기 이후 유행한 2층 수구의 형태, 조선 후기 성문과 성벽 등 성을 쌓던 기법과 공사의 순차를 연구하는 데 귀중한 자료이다.', '문의 및 안내': '', '홈페이지': 'http://www.gunwi.go.kr/tour', '주소': '대구광역시 군위군 삼국유사면 화북리 산230', '이용시간': '상시 개방', '휴일': '연중무휴', '주차': '불가능', '지정현황': '대구광역시 기념물 화산산성 (1984.05.21. 지정)', '입장료': '무료', 'review': '마을의 조형물이 예쁘긴했고 올라가며 경치는 좋았지만 특별히 볼것도 없어 조금은 허무했어요 근처 캠핑장도 있고 대구 근교라 차박도 많은것 같습니다 밤하늘 별 보기도 좋고 조용한 시골 분위기도 너무 좋네요 힐링됩니다'}
{'place': '혜원의 집', 'info': '혜원의 집은 2018년 개봉해 150만 명의 관객을 모았던 영화 ‘리틀 포레스트’ 주인공 혜원(배우 김태리)의 집이다. 군위군의 대표적인 관광지 중 한 곳인 혜원의 집은 촬영당시 모습을 그대로 보존하여 영화에서 느꼈던 감동을 그대로 느낄 수 있다. 고즈넉한 집과 마을을 거닐다 보면 어느새 나도 모르게 마음의 위로를 받을 수 있다.', '문의 및 안내': '', '홈페이지': 'https://www.gunwi.go.kr/tour/main.do', '주소': '대구광역시 군위군 우보면 미성5길 58-1', '이용시간': '09:00~18:00', '휴일':